#  Retail Analytics & Customer Segmentation Project

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Step 1: Load Raw Excel

In [ ]:
file_path = r"C:\Users\veera\OneDrive\Documents\Desktop\sales\archive\Global_Superstore2.xlsx"
xls = pd.ExcelFile(file_path)

print("Available Sheets:", xls.sheet_names)
df = xls.parse(xls.sheet_names[0])
print("Original shape:", df.shape)

# Step 2: Data Cleaning

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d-%m-%Y', errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], format='%d-%m-%Y', errors='coerce')

df['Delivery Days'] = (df['Ship Date'] - df['Order Date']).dt.days
df['Postal Code'] = df['Postal Code'].fillna(-1).astype(int).astype(str)
df.drop_duplicates(inplace=True)

numeric_cols = ['Sales', 'Profit', 'Discount', 'Quantity', 'Shipping Cost']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 3: Save Cleaned Data for build  meaningful insights in powerbi

In [ ]:
df.to_csv("global_superstore_cleaned.csv", index=False)
print("Saved: global_superstore_cleaned.csv")
print("Final shape:", df.shape)

# Step 4: Exploratory Data Analysis

In [ ]:
df['Month'] = df['Order Date'].dt.to_period('M')

# 1. Sales by Region
plt.figure(figsize=(10, 5))
df.groupby('Region')['Sales'].sum().sort_values().plot(kind='barh', color='skyblue')
plt.title("Total Sales by Region")
plt.xlabel("Sales")
plt.tight_layout()
plt.show()

# 2. Profit by Category
plt.figure(figsize=(8, 5))
df.groupby('Category')['Profit'].sum().sort_values(ascending=False).plot(kind='bar', color='lightgreen')
plt.title("Total Profit by Product Category")
plt.ylabel("Profit")
plt.tight_layout()
plt.show()

# 3. Monthly Sales Trend
plt.figure(figsize=(12, 5))
df.groupby('Month')['Sales'].sum().plot(marker='o')
plt.title("Monthly Sales Trend")
plt.ylabel("Sales")
plt.xlabel("Month")
plt.grid(True)
plt.tight_layout()
plt.show()

# 4. Discount vs Profit
plt.figure(figsize=(7, 5))
sns.scatterplot(data=df, x='Discount', y='Profit', hue='Category', alpha=0.7)
plt.title("Discount vs Profit (by Category)")
plt.tight_layout()
plt.show()

# 5. Profit vs Delivery Days
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df, x='Delivery Days', y='Profit', hue='Ship Mode', alpha=0.6)
plt.title("Profit vs Delivery Days (by Ship Mode)")
plt.xlabel("Delivery Days")
plt.ylabel("Profit")
plt.grid(True)
plt.tight_layout()
plt.show()

# 6. Top 10 Customers
top_customers = df.groupby(['Customer ID', 'Customer Name']).agg({'Sales': 'sum', 'Profit': 'sum'}).sort_values(by='Sales', ascending=False).head(10).reset_index()
plt.figure(figsize=(10, 5))
sns.barplot(data=top_customers.sort_values('Profit', ascending=False), y='Customer Name', x='Profit', palette='Greens_d', hue='Customer Name', dodge=False, legend=False)
plt.title("Top 10 Customers by Sales")
plt.xlabel("Sales")
plt.ylabel("Customer")
plt.tight_layout()
plt.show()

# Step 5: convert order date into datetime format(type casting)

In [ ]:
df = pd.read_csv("global_superstore_cleaned.csv")
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Step 6: Build RFM Table

In [ ]:
latest_date = df['Order Date'].max()
rfm = df.groupby('Customer ID').agg({
    'Order Date': lambda x: (latest_date - x.max()).days,
    'Order ID': 'nunique',
    'Sales': 'sum'
}).reset_index()
rfm.columns = ['Customer ID', 'Recency', 'Frequency', 'Monetary']

# Step 7: KMeans Clustering

In [ ]:
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

# Elbow method
inertia = []
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(rfm_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 10), inertia, marker='o')
plt.xlabel("Number of Clusters")
plt.ylabel("Inertia")
plt.title("Elbow Method for Optimal K")
plt.grid(True)
plt.tight_layout()
plt.show()

# Final model
kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Segment'] = kmeans.fit_predict(rfm_scaled)
print(rfm.groupby('Segment')[['Recency', 'Frequency', 'Monetary']].mean())

# Step 8: PCA Visualization

In [ ]:
pca = PCA(n_components=2)
rfm_pca = pca.fit_transform(rfm_scaled)
rfm['x'] = rfm_pca[:, 0]
rfm['y'] = rfm_pca[:, 1]

plt.figure(figsize=(8, 6))
sns.scatterplot(data=rfm, x='x', y='y', hue='Segment', palette='Set2', s=100)
plt.title("Customer Segments (via PCA + KMeans)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.tight_layout()
plt.show()

# Step 9: Save Segments for build dashboards

In [ ]:
rfm.to_csv("customer_segments.csv", index=False)
print("Saved: customer_segments.csv")